In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostRegressor, cv
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import datetime
%matplotlib inline


In [60]:
def make_submission(test, y_p, path):
    pd.DataFrame(list(zip(test.index, y_p))).to_csv(path, index=False, header=['unique_id', 'target'])

In [61]:
categorical_features = [
    'Организатор',
    'Дата начала подачи предложений',
    'Дата начала подачи предложений_month',
    'Дата начала подачи предложений_week',
    'Тип методу закупівлі',
    'Тип организатора',
     'Статус лота',
     'СтатусТендера', 
     'РегионОрганизатора',
     'Регион поставки',
     'Тендер',
     'Дата начала периода уточнений',
    'Дата начала периода уточнений_month',
    'Дата начала периода уточнений_week',
     'Дата завершения',
    'Дата завершения_month',
    'Дата завершения_week',
    'Дата завершения подачи предложений_month'
    'Дата завершения подачи предложений_week'
     'Дата завершения подачи предложений']

def getCatFeaturesIds(features):
    ids = []
    for i, f in enumerate(features):
        if f in categorical_features:
            ids.append(i)
        
    return ids
    

In [82]:
def date_to_dummie(data, columns):
    loc_data = data.copy()
    for col in columns:
        loc_data[col + '_month'] = pd.to_datetime(loc_data[col]).dt.month
        loc_data[col + '_week'] = pd.to_datetime(loc_data[col]).dt.weekday
        loc_data.drop(col, axis=1, inplace=True)
    return loc_data

def drop_redundant(data):
    data.drop(['Тип организатора', 'РегионОрганизатора', 'СуммаГарантииПредложенияЛота'], axis=1, inplace=True)
    return data

def split_dates(data):
    data = date_to_dummie(data, ['Дата начала подачи предложений', 'Дата завершения подачи предложений'])
    return data

def fillMissing(data):
    data.loc[data['СуммаГарантииПредложенияЛота'] == '-', "СуммаГарантииПредложенияЛота"] = 0

def preprocess_train(train_data):
#     train_data['Дата завершения'] = pd.to_datetime(train_data['Дата завершения'])
#     train_clean = train_clean[train_clean['Дата завершения'] > datetime.datetime(2017, 1, 1)]
    train_clean = train_data.drop(['IDOrganizator', 'РегионУчастника', 'ID'], axis = 1)
    train_clean = split_dates(train_clean)
    train_clean = drop_redundant(train_clean)
    return train_clean

def preprocess_test(test_data):
    test_clean = test_data.drop(['IDOrganizator'], axis = 1) 
    test_clean = split_dates(test_clean)
    test_clean = drop_redundant(test_clean)
    return test_clean

def extract_target(train_data):
    return train_data['ID']
    

In [83]:
# Read data
train_data = pd.read_csv('../data/train_agg_2.csv', index_col = 0)
train_labels = extract_target(train_data)
test_data = pd.read_csv('../data/test.csv', index_col = 0)
fillMissing(train_data)
fillMissing(test_data)

In [84]:
train_p = preprocess_train(train_data)
test_p = preprocess_test(test_data)


In [85]:
cats = getCatFeaturesIds(list(train_p.columns))

In [86]:
train_pool = Pool(train_p, train_labels, cat_features=cats)
test_pool = Pool(test_p, cat_features=cats)

In [89]:
model = CatBoostRegressor(iterations=6000, depth=5, learning_rate=0.1, loss_function="RMSE")

In [ ]:
model.fit(train_pool)

0:	learn: 2.1640871	total: 388ms	remaining: 38m 48s
1:	learn: 2.0196551	total: 508ms	remaining: 25m 22s
2:	learn: 1.8945924	total: 672ms	remaining: 22m 22s
3:	learn: 1.7868113	total: 861ms	remaining: 21m 30s
4:	learn: 1.6943080	total: 1.05s	remaining: 21m 4s
5:	learn: 1.6154923	total: 1.16s	remaining: 19m 19s
6:	learn: 1.5485775	total: 1.44s	remaining: 20m 29s
7:	learn: 1.4896530	total: 1.59s	remaining: 19m 51s
8:	learn: 1.4420663	total: 1.75s	remaining: 19m 22s
9:	learn: 1.4019074	total: 1.89s	remaining: 18m 52s
10:	learn: 1.3662128	total: 2.13s	remaining: 19m 21s
11:	learn: 1.3366468	total: 2.42s	remaining: 20m 8s
12:	learn: 1.3119632	total: 2.64s	remaining: 20m 15s
13:	learn: 1.2906509	total: 2.87s	remaining: 20m 26s
14:	learn: 1.2737733	total: 3.06s	remaining: 20m 20s
15:	learn: 1.2599858	total: 3.14s	remaining: 19m 36s
16:	learn: 1.2484957	total: 3.22s	remaining: 18m 53s
17:	learn: 1.2370184	total: 3.31s	remaining: 18m 19s
18:	learn: 1.2290481	total: 3.41s	remaining: 17m 53s
19:	l

155:	learn: 1.1283943	total: 40s	remaining: 24m 59s
156:	learn: 1.1283229	total: 40.3s	remaining: 24m 58s
157:	learn: 1.1282398	total: 40.6s	remaining: 25m
158:	learn: 1.1280886	total: 40.8s	remaining: 24m 59s
159:	learn: 1.1279088	total: 41.1s	remaining: 24m 59s
160:	learn: 1.1278192	total: 41.4s	remaining: 25m 1s
161:	learn: 1.1276592	total: 41.7s	remaining: 25m 1s
162:	learn: 1.1275316	total: 41.9s	remaining: 24m 59s
163:	learn: 1.1272353	total: 42.2s	remaining: 25m
164:	learn: 1.1270049	total: 42.5s	remaining: 25m 2s
165:	learn: 1.1267688	total: 42.8s	remaining: 25m 4s
166:	learn: 1.1266001	total: 43.1s	remaining: 25m 6s
167:	learn: 1.1265070	total: 43.4s	remaining: 25m 5s
168:	learn: 1.1264256	total: 43.7s	remaining: 25m 7s
169:	learn: 1.1262230	total: 44s	remaining: 25m 9s
170:	learn: 1.1261097	total: 44.3s	remaining: 25m 8s
171:	learn: 1.1260108	total: 44.6s	remaining: 25m 10s
172:	learn: 1.1257481	total: 44.8s	remaining: 25m 9s
173:	learn: 1.1256107	total: 45.1s	remaining: 25m 

307:	learn: 1.1141116	total: 1m 23s	remaining: 25m 47s
308:	learn: 1.1140785	total: 1m 24s	remaining: 25m 49s
309:	learn: 1.1139299	total: 1m 24s	remaining: 25m 48s
310:	learn: 1.1138939	total: 1m 24s	remaining: 25m 48s
311:	learn: 1.1138418	total: 1m 25s	remaining: 25m 50s
312:	learn: 1.1136773	total: 1m 25s	remaining: 25m 50s
313:	learn: 1.1135964	total: 1m 25s	remaining: 25m 49s
314:	learn: 1.1135443	total: 1m 25s	remaining: 25m 51s
315:	learn: 1.1135273	total: 1m 26s	remaining: 25m 52s
316:	learn: 1.1134858	total: 1m 26s	remaining: 25m 52s
317:	learn: 1.1134562	total: 1m 26s	remaining: 25m 51s
318:	learn: 1.1133371	total: 1m 27s	remaining: 25m 50s
319:	learn: 1.1131807	total: 1m 27s	remaining: 25m 48s
320:	learn: 1.1131073	total: 1m 27s	remaining: 25m 47s
321:	learn: 1.1130010	total: 1m 27s	remaining: 25m 46s
322:	learn: 1.1128948	total: 1m 27s	remaining: 25m 45s
323:	learn: 1.1128583	total: 1m 28s	remaining: 25m 43s
324:	learn: 1.1128128	total: 1m 28s	remaining: 25m 45s
325:	learn

457:	learn: 1.1073395	total: 2m 7s	remaining: 25m 37s
458:	learn: 1.1073268	total: 2m 7s	remaining: 25m 37s
459:	learn: 1.1072696	total: 2m 7s	remaining: 25m 37s
460:	learn: 1.1072561	total: 2m 8s	remaining: 25m 38s
461:	learn: 1.1071897	total: 2m 8s	remaining: 25m 37s
462:	learn: 1.1071687	total: 2m 8s	remaining: 25m 37s
463:	learn: 1.1071176	total: 2m 8s	remaining: 25m 37s
464:	learn: 1.1070463	total: 2m 9s	remaining: 25m 36s
465:	learn: 1.1070269	total: 2m 9s	remaining: 25m 36s
466:	learn: 1.1070024	total: 2m 9s	remaining: 25m 35s
467:	learn: 1.1069791	total: 2m 9s	remaining: 25m 35s
468:	learn: 1.1069302	total: 2m 10s	remaining: 25m 35s
469:	learn: 1.1068526	total: 2m 10s	remaining: 25m 34s
470:	learn: 1.1068421	total: 2m 10s	remaining: 25m 34s
471:	learn: 1.1068062	total: 2m 10s	remaining: 25m 33s
472:	learn: 1.1067916	total: 2m 11s	remaining: 25m 34s
473:	learn: 1.1067411	total: 2m 11s	remaining: 25m 34s
474:	learn: 1.1067257	total: 2m 11s	remaining: 25m 34s
475:	learn: 1.1066436

607:	learn: 1.1023415	total: 2m 51s	remaining: 25m 23s
608:	learn: 1.1023241	total: 2m 52s	remaining: 25m 23s
609:	learn: 1.1023215	total: 2m 52s	remaining: 25m 22s
610:	learn: 1.1022978	total: 2m 52s	remaining: 25m 23s
611:	learn: 1.1021374	total: 2m 52s	remaining: 25m 22s
612:	learn: 1.1020792	total: 2m 53s	remaining: 25m 22s
613:	learn: 1.1020376	total: 2m 53s	remaining: 25m 23s
614:	learn: 1.1020284	total: 2m 53s	remaining: 25m 23s
615:	learn: 1.1020151	total: 2m 54s	remaining: 25m 23s
616:	learn: 1.1019963	total: 2m 54s	remaining: 25m 23s
617:	learn: 1.1019792	total: 2m 54s	remaining: 25m 23s
618:	learn: 1.1019720	total: 2m 55s	remaining: 25m 24s
619:	learn: 1.1019338	total: 2m 55s	remaining: 25m 25s
620:	learn: 1.1018729	total: 2m 56s	remaining: 25m 24s
621:	learn: 1.1018513	total: 2m 56s	remaining: 25m 24s
622:	learn: 1.1018335	total: 2m 56s	remaining: 25m 24s
623:	learn: 1.1018122	total: 2m 56s	remaining: 25m 24s
624:	learn: 1.1017869	total: 2m 57s	remaining: 25m 24s
625:	learn

...

3111:	learn: 1.0698374	total: 15m 46s	remaining: 14m 38s
3112:	learn: 1.0698296	total: 15m 46s	remaining: 14m 37s
3113:	learn: 1.0698063	total: 15m 47s	remaining: 14m 37s
3114:	learn: 1.0698003	total: 15m 47s	remaining: 14m 37s
3115:	learn: 1.0697946	total: 15m 47s	remaining: 14m 37s
3116:	learn: 1.0697877	total: 15m 48s	remaining: 14m 36s
3117:	learn: 1.0697516	total: 15m 48s	remaining: 14m 36s
3118:	learn: 1.0697442	total: 15m 48s	remaining: 14m 36s
3119:	learn: 1.0697357	total: 15m 49s	remaining: 14m 36s
3120:	learn: 1.0697288	total: 15m 49s	remaining: 14m 35s
3121:	learn: 1.0697195	total: 15m 49s	remaining: 14m 35s
3122:	learn: 1.0697131	total: 15m 50s	remaining: 14m 35s
3123:	learn: 1.0697017	total: 15m 50s	remaining: 14m 35s
3124:	learn: 1.0696988	total: 15m 50s	remaining: 14m 34s
3125:	learn: 1.0696904	total: 15m 51s	remaining: 14m 34s
3126:	learn: 1.0696902	total: 15m 51s	remaining: 14m 34s
3127:	learn: 1.0696842	total: 15m 51s	remaining: 14m 33s
3128:	learn: 1.0696740	total: 1

3257:	learn: 1.0686435	total: 16m 35s	remaining: 13m 58s
3258:	learn: 1.0686356	total: 16m 36s	remaining: 13m 57s
3259:	learn: 1.0686311	total: 16m 36s	remaining: 13m 57s
3260:	learn: 1.0686222	total: 16m 36s	remaining: 13m 57s
3261:	learn: 1.0686103	total: 16m 37s	remaining: 13m 56s
3262:	learn: 1.0686034	total: 16m 37s	remaining: 13m 56s
3263:	learn: 1.0686019	total: 16m 37s	remaining: 13m 56s
3264:	learn: 1.0685784	total: 16m 38s	remaining: 13m 56s
3265:	learn: 1.0685626	total: 16m 38s	remaining: 13m 55s
3266:	learn: 1.0685540	total: 16m 38s	remaining: 13m 55s
3267:	learn: 1.0685461	total: 16m 39s	remaining: 13m 55s
3268:	learn: 1.0685391	total: 16m 39s	remaining: 13m 54s
3269:	learn: 1.0685294	total: 16m 39s	remaining: 13m 54s
3270:	learn: 1.0685232	total: 16m 40s	remaining: 13m 54s
3271:	learn: 1.0685172	total: 16m 40s	remaining: 13m 54s
3272:	learn: 1.0685082	total: 16m 40s	remaining: 13m 53s
3273:	learn: 1.0685022	total: 16m 41s	remaining: 13m 53s
3274:	learn: 1.0684978	total: 1

3402:	learn: 1.0675915	total: 17m 22s	remaining: 13m 15s
3403:	learn: 1.0675882	total: 17m 22s	remaining: 13m 15s
3404:	learn: 1.0675275	total: 17m 23s	remaining: 13m 15s
3405:	learn: 1.0675231	total: 17m 23s	remaining: 13m 14s
3406:	learn: 1.0675190	total: 17m 23s	remaining: 13m 14s
3407:	learn: 1.0675125	total: 17m 24s	remaining: 13m 14s
3408:	learn: 1.0674886	total: 17m 24s	remaining: 13m 13s
3409:	learn: 1.0674840	total: 17m 24s	remaining: 13m 13s
3410:	learn: 1.0674727	total: 17m 25s	remaining: 13m 13s
3411:	learn: 1.0674610	total: 17m 25s	remaining: 13m 12s
3412:	learn: 1.0674582	total: 17m 25s	remaining: 13m 12s
3413:	learn: 1.0674539	total: 17m 26s	remaining: 13m 12s
3414:	learn: 1.0674463	total: 17m 26s	remaining: 13m 12s
3415:	learn: 1.0674424	total: 17m 26s	remaining: 13m 11s
3416:	learn: 1.0674363	total: 17m 27s	remaining: 13m 11s
3417:	learn: 1.0674266	total: 17m 27s	remaining: 13m 11s
3418:	learn: 1.0674220	total: 17m 27s	remaining: 13m 10s
3419:	learn: 1.0674173	total: 1

3548:	learn: 1.0664266	total: 18m 10s	remaining: 12m 33s
3549:	learn: 1.0663987	total: 18m 11s	remaining: 12m 33s
3550:	learn: 1.0663932	total: 18m 11s	remaining: 12m 32s
3551:	learn: 1.0663906	total: 18m 11s	remaining: 12m 32s
3552:	learn: 1.0663683	total: 18m 12s	remaining: 12m 32s
3553:	learn: 1.0663588	total: 18m 12s	remaining: 12m 31s
3554:	learn: 1.0663510	total: 18m 12s	remaining: 12m 31s
3555:	learn: 1.0663487	total: 18m 13s	remaining: 12m 31s
3556:	learn: 1.0663431	total: 18m 13s	remaining: 12m 30s
3557:	learn: 1.0663391	total: 18m 13s	remaining: 12m 30s
3558:	learn: 1.0663211	total: 18m 14s	remaining: 12m 30s
3559:	learn: 1.0663177	total: 18m 14s	remaining: 12m 30s
3560:	learn: 1.0663053	total: 18m 14s	remaining: 12m 29s
3561:	learn: 1.0662958	total: 18m 15s	remaining: 12m 29s
3562:	learn: 1.0662930	total: 18m 15s	remaining: 12m 29s
3563:	learn: 1.0662867	total: 18m 15s	remaining: 12m 28s
3564:	learn: 1.0662840	total: 18m 16s	remaining: 12m 28s
3565:	learn: 1.0662807	total: 1

3693:	learn: 1.0652306	total: 18m 59s	remaining: 11m 51s
3694:	learn: 1.0652266	total: 18m 59s	remaining: 11m 50s
3695:	learn: 1.0652212	total: 18m 59s	remaining: 11m 50s
3696:	learn: 1.0652151	total: 19m	remaining: 11m 50s
3697:	learn: 1.0652102	total: 19m	remaining: 11m 50s
3698:	learn: 1.0652022	total: 19m	remaining: 11m 49s
3699:	learn: 1.0651932	total: 19m 1s	remaining: 11m 49s
3700:	learn: 1.0651899	total: 19m 1s	remaining: 11m 49s
3701:	learn: 1.0651817	total: 19m 1s	remaining: 11m 48s
3702:	learn: 1.0651724	total: 19m 2s	remaining: 11m 48s
3703:	learn: 1.0651642	total: 19m 2s	remaining: 11m 48s
3704:	learn: 1.0651603	total: 19m 2s	remaining: 11m 47s
3705:	learn: 1.0651551	total: 19m 3s	remaining: 11m 47s
3706:	learn: 1.0651525	total: 19m 3s	remaining: 11m 47s
3707:	learn: 1.0651454	total: 19m 3s	remaining: 11m 46s
3708:	learn: 1.0651417	total: 19m 4s	remaining: 11m 46s
3709:	learn: 1.0651400	total: 19m 4s	remaining: 11m 46s
3710:	learn: 1.0651335	total: 19m 4s	remaining: 11m 46

In [50]:
y_p = model.predict(test_pool)

In [16]:
params = {'iterations': 50, 'depth': 2, 'loss_function': 'RMSE', 'learning_rate':1 }

In [27]:
# cv(train_pool, params=params)

In [51]:
make_submission(test_p, y_p, '~/results/trash.csv')